# Window Size 5

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import os

def diebold_mariano_test(actual, pred1, pred2, h=1):
    """
    Diebold-Mariano test using Mean Squared Error loss function with Newey-West

    H0: E[dt] = 0 (equal prediction accuracy)
    H1: E[dt] ≠ 0 (unequal prediction accuracy)

    Test statistic: DM = d̄ / σ̂_d̄ ~ N(0,1)
    """

    # Align arrays
    min_len = min(len(actual), len(pred1), len(pred2))
    actual = np.array(actual)[:min_len]
    pred1 = np.array(pred1)[:min_len]
    pred2 = np.array(pred2)[:min_len]

    # Remove missing values
    mask = ~(np.isnan(actual) | np.isnan(pred1) | np.isnan(pred2))
    actual = actual[mask]
    pred1 = pred1[mask]
    pred2 = pred2[mask]

    if len(actual) < 10:
        return np.nan, np.nan

    # Calculate squared forecast errors
    e1_squared = (actual - pred1)**2
    e2_squared = (actual - pred2)**2

    # Loss differential: dt = e²1,t - e²2,t
    d = e1_squared - e2_squared
    d_mean = np.mean(d)
    n = len(d)

    # Newey-West variance estimator
    def newey_west_variance(d, h):
        n = len(d)
        d_centered = d - np.mean(d)

        # Bandwidth selection
        bandwidth = max(1, int(4 * (n/100)**(2/9)))

        # Calculate autocovariances with Bartlett kernel
        gamma_0 = np.mean(d_centered**2)
        gamma_sum = 0

        for k in range(1, min(bandwidth + 1, n)):
            if k < n:
                weight = 1 - k / (bandwidth + 1)
                gamma_k = np.mean(d_centered[k:] * d_centered[:-k])
                gamma_sum += 2 * weight * gamma_k

        return gamma_0 + gamma_sum

    # Calculate long-run variance
    long_run_var = newey_west_variance(d, h)

    if long_run_var <= 0:
        long_run_var = np.var(d, ddof=1)
        if long_run_var <= 0:
            return np.nan, np.nan

    # DM statistic
    dm_stat = d_mean / np.sqrt(long_run_var / n)

    if np.isnan(dm_stat) or np.isinf(dm_stat):
        return np.nan, np.nan

    # Two-tailed p-value
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))

    return dm_stat, p_value


def run_pairwise_dm_tests(actual, predictions_dict):
    """
    Run pairwise DM tests for all model combinations
    """
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)

    # Initialize matrices
    dm_stats = np.full((n_models, n_models), np.nan)
    p_values = np.full((n_models, n_models), np.nan)

    print(f"Running pairwise DM tests...")

    # Pairwise comparisons
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:  # Skip diagonal
                pred1 = predictions_dict[model_a]
                pred2 = predictions_dict[model_b]

                dm_stat, p_value = diebold_mariano_test(actual, pred1, pred2)

                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_value

    # Create DataFrames
    dm_df = pd.DataFrame(dm_stats, index=model_names, columns=model_names)
    p_df = pd.DataFrame(p_values, index=model_names, columns=model_names)

    return dm_df, p_df


def format_pvalue_table(p_df):
    """Format p-values with significance indicators"""
    formatted = p_df.copy().astype(object)

    for i in range(len(p_df)):
        for j in range(len(p_df.columns)):
            if i == j:
                formatted.iloc[i, j] = "-"
            elif i > j:  # Lower triangle - leave empty
                formatted.iloc[i, j] = ""
            elif not np.isnan(p_df.iloc[i, j]):
                p_val = p_df.iloc[i, j]

                if p_val < 0.001:
                    formatted.iloc[i, j] = f"{p_val:.6f}***"
                elif p_val < 0.01:
                    formatted.iloc[i, j] = f"{p_val:.6f}**"
                elif p_val < 0.05:
                    formatted.iloc[i, j] = f"{p_val:.6f}*"
                else:
                    formatted.iloc[i, j] = f"{p_val:.6f}"
            else:
                formatted.iloc[i, j] = ""

    return formatted


def format_dm_table(dm_df, p_df):
    """Format DM statistics table with significance indicators based on p-values"""
    formatted = dm_df.copy().astype(object)

    for i in range(len(dm_df)):
        for j in range(len(dm_df.columns)):
            if i == j:
                formatted.iloc[i, j] = "-"
            elif i > j:
                formatted.iloc[i, j] = ""
            elif not (np.isnan(dm_df.iloc[i, j]) or np.isnan(p_df.iloc[i, j])):
                dm_val = dm_df.iloc[i, j]
                p_val = p_df.iloc[i, j]

                # Add significance stars based on p-values
                if p_val < 0.001:
                    stars = "***"
                elif p_val < 0.01:
                    stars = "**"
                elif p_val < 0.05:
                    stars = "*"
                else:
                    stars = ""

                formatted.iloc[i, j] = f"{dm_val:.4f}{stars}"
            else:
                formatted.iloc[i, j] = ""

    return formatted


def create_dm_heatmap(p_matrix, title="Diebold-Mariano Test P-Values"):
    """
    Create heatmap with black diagonal X marks and pure black for p >= 0.05
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Prepare data matrix
    n_models = len(p_matrix)
    heatmap_data = np.full((n_models, n_models), np.nan)

    # Fill data: 0.051 for diagonal and p>=0.05, actual p-values for p<0.05
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
            else:
                p_val = p_matrix.iloc[i, j]
                if np.isnan(p_val) or p_val >= 0.05:
                    heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
                else:
                    heatmap_data[i, j] = p_val  # Keep actual p-value for coloring

    # Create discrete colormap with explicit black at the end
    colors = ['#003300', '#006600', '#009900', '#00CC00', '#33FF33', '#66FF66', '#99FF99',
              '#CCFF99', '#FFFF66', '#FFCC33', '#FF9900', '#FF6600', '#FF3300', '#CC0000', '#000000']

    # Create boundaries for discrete colors
    boundaries = np.linspace(0, 0.05, len(colors)-1).tolist() + [0.051, 1.0]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries, len(colors))

    # Plot heatmap
    im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='equal', interpolation='nearest')

    # Add white X marks on diagonal
    for i in range(n_models):
        ax.text(i, i, 'X', ha='center', va='center',
                color='white', fontsize=18, fontweight='bold')

    # Add white grid lines
    for i in range(n_models + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1.0)
        ax.axvline(i - 0.5, color='white', linewidth=1.0)

    # Set labels
    model_names = p_matrix.columns.tolist()
    ax.set_xticks(range(n_models))
    ax.set_yticks(range(n_models))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=11, color='black')
    ax.set_yticklabels(model_names, fontsize=11, color='black')

    # Add title
    ax.set_title(title + '\n(Window 5)', fontsize=18, fontweight='bold', color='black', pad=30)

    # Create colorbar with discrete ticks including black
    cbar = plt.colorbar(im, ax=ax, shrink=0.7, aspect=25, pad=0.02, boundaries=boundaries, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.051])
    cbar.set_label('P-Value', fontsize=14, color='black', labelpad=15)
    cbar.ax.tick_params(labelsize=12, colors='black')

    # Set colorbar labels with black indicator
    cbar_labels = ['0.00', '0.01', '0.02', '0.03', '0.04', '0.05', '≥0.05']
    cbar.set_ticklabels(cbar_labels)

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove tick marks but keep labels
    ax.tick_params(length=0, colors='black')

    # Adjust layout
    plt.subplots_adjust(bottom=0.08, top=0.92, left=0.1, right=0.88)

    plt.savefig('dm_test_heatmap_fixed.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()

    return ax


def run_dm_test_analysis():
    """
    Load data and run DM tests for all model pairs
    """

    # Input folder: DM_Test_Data
    DATA_DIR = os.path.join(os.getcwd(), "DM_Test_Data")

    # Model files
    model_files = {
        "OLS": "ols_results_lag5_full.csv",
        "LASSO": "lasso_results_lag5_full.csv",
        "Ridge": "ridge_results_lag5_full.csv",
        "ElasticNet": "enet_results_lag5_full.csv",
        "PCR": "pcr_results_lag5_full.csv",
        "GLM": "glm_results_lag5_full.csv",
        "RF": "rf_results_lag5_full.csv",
        "GBRT": "gbrt_results_lag5_full.csv",
        "NN1": "nn1_results_lag5_full.csv",
        "NN2": "nn2_results_lag5_full.csv",
        "NN3": "nn3_results_lag5_full.csv",
        "NN4": "nn4_results_lag5_full.csv",
        "NN5": "nn5_results_lag5_full.csv",
        "Chronos-T5 Tiny": "chronost5tiny_results_lag5_full.csv",
        "Chronos-Bolt Tiny": "chronosbolttiny_results_lag5_full.csv",
        "Chronos-Bolt Mini": "chronosboltmini_results_lag5_full.csv",
        "Chronos-Bolt Small": "chronosboltsmall_results_lag5_full.csv",
        "TimesFM 1.0-200M": "timesfm1_results_lag5_full.csv",
        "TimesFM 2.0-500M": "timesfm2_results_lag5_full.csv",
        "Moirai Small": "uni2tssmall_results_lag5_full.csv",
        "Moirai-Moe Small": "uni2tssmallmoe_results_lag5_full.csv",
        "Moirai-Moe Base": "uni2tsbasemoe_results_lag5_full.csv"
    }

    print("DIEBOLD-MARIANO TEST")
    print("="*60)
    print("Loss Function: Mean Squared Error (MSE)")
    print("H₀: E[dₜ] = 0 (equal prediction accuracy)")
    print("H₁: E[dₜ] ≠ 0 (unequal prediction accuracy)")
    print("Variance Estimator: Newey-West with automatic bandwidth")
    print("="*60)

    # Load and align data
    dfs = []
    model_names = []

    for model_name, file_name in model_files.items():
        file_path = os.path.join(DATA_DIR, file_name)
        try:
            df = pd.read_csv(file_path)

            # Standardize column names
            if 'excess_ret' in df.columns:
                df = df.rename(columns={'excess_ret': 'actual'})
            elif 'y_true' in df.columns:
                df = df.rename(columns={'y_true': 'actual'})

            if 'y_pred' in df.columns:
                df = df.rename(columns={'y_pred': 'predicted'})
            elif 'predicted_excess_returns_lag5' in df.columns:
                df = df.rename(columns={'predicted_excess_returns_lag5': 'predicted'})

            dfs.append(df[['actual', 'predicted']])
            model_names.append(model_name)
            print(f"Loaded: {model_name}")

        except FileNotFoundError:
            print(f"Warning: {file_path} not found, skipping {model_name}")

    # Align to common length
    min_length = min(len(df) for df in dfs)
    print(f"\nSample size after alignment: {min_length}")

    actual_values = dfs[0]['actual'].iloc[:min_length].values
    predictions = {}

    for df, name in zip(dfs, model_names):
        predictions[name] = df['predicted'].iloc[:min_length].values

    print(f"Models included: {len(model_names)}")

    # Run DM tests
    dm_matrix, p_matrix = run_pairwise_dm_tests(actual_values, predictions)

    # Format results
    formatted_dm = format_dm_table(dm_matrix, p_matrix)
    formatted_p = format_pvalue_table(p_matrix)

    print("\nDM TEST STATISTICS:")
    print("Interpretation:")
    print("- DM > 0: Row model has HIGHER MSE (worse) than Column model")
    print("- DM < 0: Row model has LOWER MSE (better) than Column model")
    print("- Significance: *** p<0.001, ** p<0.01, * p<0.05")
    print("- p > 0.05: No significant difference (accept H₀)")
    print("-" * 80)
    print(formatted_dm.to_string())

    print("\n\nP-VALUES:")
    print("- p < 0.05: Reject H₀ (significant difference in accuracy)")
    print("- p ≥ 0.05: Accept H₀ (no significant difference)")
    print("-" * 80)
    print(formatted_p.to_string())

    # Save the DM test statistics results to a CSV file
    formatted_dm.to_csv("dm_statistics_formatted.csv", index=True)
    p_matrix.to_csv("dm_pvalues_formatted.csv", index=True)

    # Create summary of best performing models
    mse_performance = {}
    for name, preds in predictions.items():
        mse = np.mean((actual_values - preds)**2)
        mse_performance[name] = mse

    # Summary of significant outperformances
    print("\n\nSUMMARY OF SIGNIFICANT OUTPERFORMANCES:")
    print("-" * 50)

    significant_better = {}
    for i, model_a in enumerate(model_names):
        significant_better[model_a] = []
        for j, model_b in enumerate(model_names):
            if i != j:
                dm_stat = dm_matrix.iloc[i, j]
                p_val = p_matrix.iloc[i, j]

                # Model A significantly better than Model B (DM < 0 and p < 0.05)
                if not np.isnan(p_val) and p_val < 0.05 and dm_stat < 0:
                    significant_better[model_a].append(model_b)

    for model, beaten_models in significant_better.items():
        if beaten_models:
            print(f"{model} significantly outperforms: {', '.join(beaten_models[:5])}")
            if len(beaten_models) > 5:
                print(f"   ... and {len(beaten_models)-5} more models")

    # Create and display heatmap
    print("\nGenerating DM test heatmap...")
    create_dm_heatmap(p_matrix)

    print(f"\nResults saved:")
    print("- dm_statistics5.csv")
    print("- dm_pvalues.csv")
    print("- dm_statistics_formatted5.csv")
    print("- dm_pvalues_formatted.csv")
    print("- dm_test_heatmap_fixed.png")

    return dm_matrix, p_matrix

def run_pairwise_dm_tests(actual, predictions_dict):
    """
    Run pairwise DM tests for all model combinations
    """
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)

    # Initialize matrices
    dm_stats = np.full((n_models, n_models), np.nan)
    p_values = np.full((n_models, n_models), np.nan)

    print(f"Running pairwise DM tests...")

    # Pairwise comparisons
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:  # Skip diagonal
                pred1 = predictions_dict[model_a]
                pred2 = predictions_dict[model_b]

                dm_stat, p_value = diebold_mariano_test(actual, pred1, pred2)

                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_value

    # Create DataFrames
    dm_df = pd.DataFrame(dm_stats, index=model_names, columns=model_names)
    p_df = pd.DataFrame(p_values, index=model_names, columns=model_names)

    return dm_df, p_df

def create_dm_heatmap(p_matrix, title="Diebold-Mariano Test P-Values"):
    """
    Create heatmap with black diagonal X marks and pure black for p >= 0.05
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Prepare data matrix
    n_models = len(p_matrix)
    heatmap_data = np.full((n_models, n_models), np.nan)

    # Fill data: 0.051 for diagonal and p>=0.05, actual p-values for p<0.05
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
            else:
                p_val = p_matrix.iloc[i, j]
                if np.isnan(p_val) or p_val >= 0.05:
                    heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
                else:
                    heatmap_data[i, j] = p_val  # Keep actual p-value for coloring

    # Create discrete colormap with explicit black at the end
    colors = ['#006400', '#007400', '#009900', '#00CC00', '#33FF33', '#66FF66', '#99FF99',
              '#CCFF99', '#FFFF66', '#FFCC33', '#FF9900', '#FF6600', '#FF3300', '#CC0000', '#000000']

    # Create boundaries for discrete colors
    boundaries = np.linspace(0, 0.05, len(colors)-1).tolist() + [0.051, 1.0]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries, len(colors))

    # Plot heatmap
    im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='equal', interpolation='nearest')

    # Add white X marks on diagonal
    for i in range(n_models):
        ax.text(i, i, 'X', ha='center', va='center',
                color='white', fontsize=18, fontweight='bold')

    # Add white grid lines
    for i in range(n_models + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1.0)
        ax.axvline(i - 0.5, color='white', linewidth=1.0)

    # Set labels
    model_names = p_matrix.columns.tolist()
    ax.set_xticks(range(n_models))
    ax.set_yticks(range(n_models))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=11, color='black')
    ax.set_yticklabels(model_names, fontsize=11, color='black')

    # Add title
    ax.set_title(title + '\n(Window 5)', fontsize=18, fontweight='bold', color='black', pad=30)

    # Create colorbar with discrete ticks including black
    cbar = plt.colorbar(im, ax=ax, shrink=0.7, aspect=25, pad=0.02, boundaries=boundaries, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.051])
    cbar.set_label('P-Value', fontsize=14, color='black', labelpad=15)
    cbar.ax.tick_params(labelsize=12, colors='black')

    # Set colorbar labels with black indicator
    cbar_labels = ['0.00', '0.01', '0.02', '0.03', '0.04', '0.05', '≥0.05']
    cbar.set_ticklabels(cbar_labels)

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove tick marks but keep labels
    ax.tick_params(length=0, colors='black')

    # Adjust layout
    plt.subplots_adjust(bottom=0.08, top=0.92, left=0.1, right=0.88)

    plt.savefig('dm_test_heatmap_fixed.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()

    return ax

# Run the analysis
if __name__ == "__main__":
    dm_df, p_df = run_dm_test_analysis()

# Window Size 21

In [ ]:
def diebold_mariano_test(actual, pred1, pred2, h=1):
    """
    Diebold-Mariano test using Mean Squared Error loss function with Newey-West

    H0: E[dt] = 0 (equal prediction accuracy)
    H1: E[dt] ≠ 0 (unequal prediction accuracy)

    Test statistic: DM = d̄ / σ̂_d̄ ~ N(0,1)
    """

    # Align arrays
    min_len = min(len(actual), len(pred1), len(pred2))
    actual = np.array(actual)[:min_len]
    pred1 = np.array(pred1)[:min_len]
    pred2 = np.array(pred2)[:min_len]

    # Remove missing values
    mask = ~(np.isnan(actual) | np.isnan(pred1) | np.isnan(pred2))
    actual = actual[mask]
    pred1 = pred1[mask]
    pred2 = pred2[mask]

    if len(actual) < 10:
        return np.nan, np.nan

    # Calculate squared forecast errors
    e1_squared = (actual - pred1)**2
    e2_squared = (actual - pred2)**2

    # Loss differential: dt = e²1,t - e²2,t
    d = e1_squared - e2_squared
    d_mean = np.mean(d)
    n = len(d)

    # Newey-West variance estimator
    def newey_west_variance(d, h):
        n = len(d)
        d_centered = d - np.mean(d)

        # Bandwidth selection
        bandwidth = max(1, int(4 * (n/100)**(2/9)))

        # Calculate autocovariances with Bartlett kernel
        gamma_0 = np.mean(d_centered**2)
        gamma_sum = 0

        for k in range(1, min(bandwidth + 1, n)):
            if k < n:
                weight = 1 - k / (bandwidth + 1)
                gamma_k = np.mean(d_centered[k:] * d_centered[:-k])
                gamma_sum += 2 * weight * gamma_k

        return gamma_0 + gamma_sum

    # Calculate long-run variance
    long_run_var = newey_west_variance(d, h)

    if long_run_var <= 0:
        long_run_var = np.var(d, ddof=1)
        if long_run_var <= 0:
            return np.nan, np.nan

    # DM statistic
    dm_stat = d_mean / np.sqrt(long_run_var / n)

    if np.isnan(dm_stat) or np.isinf(dm_stat):
        return np.nan, np.nan

    # Two-tailed p-value
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))

    return dm_stat, p_value


def run_pairwise_dm_tests(actual, predictions_dict):
    """
    Run pairwise DM tests for all model combinations
    """
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)

    # Initialize matrices
    dm_stats = np.full((n_models, n_models), np.nan)
    p_values = np.full((n_models, n_models), np.nan)

    print(f"Running pairwise DM tests...")

    # Pairwise comparisons
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:  # Skip diagonal
                pred1 = predictions_dict[model_a]
                pred2 = predictions_dict[model_b]

                dm_stat, p_value = diebold_mariano_test(actual, pred1, pred2)

                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_value

    # Create DataFrames
    dm_df = pd.DataFrame(dm_stats, index=model_names, columns=model_names)
    p_df = pd.DataFrame(p_values, index=model_names, columns=model_names)

    return dm_df, p_df


def format_pvalue_table(p_df):
    """Format p-values with significance indicators"""
    formatted = p_df.copy().astype(object)

    for i in range(len(p_df)):
        for j in range(len(p_df.columns)):
            if i == j:
                formatted.iloc[i, j] = "-"
            elif i > j:  # Lower triangle - leave empty
                formatted.iloc[i, j] = ""
            elif not np.isnan(p_df.iloc[i, j]):
                p_val = p_df.iloc[i, j]

                if p_val < 0.001:
                    formatted.iloc[i, j] = f"{p_val:.6f}***"
                elif p_val < 0.01:
                    formatted.iloc[i, j] = f"{p_val:.6f}**"
                elif p_val < 0.05:
                    formatted.iloc[i, j] = f"{p_val:.6f}*"
                else:
                    formatted.iloc[i, j] = f"{p_val:.6f}"
            else:
                formatted.iloc[i, j] = ""

    return formatted


def format_dm_table(dm_df, p_df):
    """Format DM statistics table with significance indicators based on p-values"""
    formatted = dm_df.copy().astype(object)

    for i in range(len(dm_df)):
        for j in range(len(dm_df.columns)):
            if i == j:
                formatted.iloc[i, j] = "-"
            elif i > j:
                formatted.iloc[i, j] = ""
            elif not (np.isnan(dm_df.iloc[i, j]) or np.isnan(p_df.iloc[i, j])):
                dm_val = dm_df.iloc[i, j]
                p_val = p_df.iloc[i, j]

                # Add significance stars based on p-values
                if p_val < 0.001:
                    stars = "***"
                elif p_val < 0.01:
                    stars = "**"
                elif p_val < 0.05:
                    stars = "*"
                else:
                    stars = ""

                formatted.iloc[i, j] = f"{dm_val:.4f}{stars}"
            else:
                formatted.iloc[i, j] = ""

    return formatted


def create_dm_heatmap(p_matrix, title="Diebold-Mariano Test P-Values"):
    """
    Create heatmap with black diagonal X marks and pure black for p >= 0.05
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Prepare data matrix
    n_models = len(p_matrix)
    heatmap_data = np.full((n_models, n_models), np.nan)

    # Fill data: 0.051 for diagonal and p>=0.05, actual p-values for p<0.05
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
            else:
                p_val = p_matrix.iloc[i, j]
                if np.isnan(p_val) or p_val >= 0.05:
                    heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
                else:
                    heatmap_data[i, j] = p_val  # Keep actual p-value for coloring

    # Create discrete colormap with explicit black at the end
    colors = ['#003300', '#006600', '#009900', '#00CC00', '#33FF33', '#66FF66', '#99FF99',
              '#CCFF99', '#FFFF66', '#FFCC33', '#FF9900', '#FF6600', '#FF3300', '#CC0000', '#000000']

    # Create boundaries for discrete colors
    boundaries = np.linspace(0, 0.05, len(colors)-1).tolist() + [0.051, 1.0]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries, len(colors))

    # Plot heatmap
    im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='equal', interpolation='nearest')

    # Add white X marks on diagonal
    for i in range(n_models):
        ax.text(i, i, 'X', ha='center', va='center',
                color='white', fontsize=18, fontweight='bold')

    # Add white grid lines
    for i in range(n_models + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1.0)
        ax.axvline(i - 0.5, color='white', linewidth=1.0)

    # Set labels
    model_names = p_matrix.columns.tolist()
    ax.set_xticks(range(n_models))
    ax.set_yticks(range(n_models))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=11, color='black')
    ax.set_yticklabels(model_names, fontsize=11, color='black')

    # Add title
    ax.set_title(title + '\n(Window 21)', fontsize=18, fontweight='bold', color='black', pad=30)

    # Create colorbar with discrete ticks including black
    cbar = plt.colorbar(im, ax=ax, shrink=0.7, aspect=25, pad=0.02, boundaries=boundaries, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.051])
    cbar.set_label('P-Value', fontsize=14, color='black', labelpad=15)
    cbar.ax.tick_params(labelsize=12, colors='black')

    # Set colorbar labels with black indicator
    cbar_labels = ['0.00', '0.01', '0.02', '0.03', '0.04', '0.05', '≥0.05']
    cbar.set_ticklabels(cbar_labels)

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove tick marks but keep labels
    ax.tick_params(length=0, colors='black')

    # Adjust layout
    plt.subplots_adjust(bottom=0.08, top=0.92, left=0.1, right=0.88)

    plt.savefig('dm_test_heatmap_fixed.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()

    return ax


def run_dm_test_analysis():
    """
    Load data and run DM tests for all model pairs
    """

    # Input folder: DM_Test_Data
    DATA_DIR = os.path.join(os.getcwd(), "DM_Test_Data")

    # Model files
    model_files = {
        "OLS": "ols_results_lag21_full.csv",
        "LASSO": "lasso_results_lag21_full.csv",
        "Ridge": "ridge_results_lag21_full.csv",
        "ElasticNet": "enet_results_lag21_full.csv",
        "PCR": "pcr_results_lag21_full.csv",
        "GLM": "glm_results_lag21_full.csv",
        "RF": "rf_results_lag21_full.csv",
        "GBRT": "gbrt_results_lag21_full.csv",
        "NN1": "nn1_results_lag21_full.csv",
        "NN2": "nn2_results_lag21_full.csv",
        "NN3": "nn3_results_lag21_full.csv",
        "NN4": "nn4_results_lag21_full.csv",
        "NN5": "nn5_results_lag21_full.csv",
        "Chronos-T5 Tiny": "chronost5tiny_results_lag21_full.csv",
        "Chronos-Bolt Tiny": "chronosbolttiny_results_lag21_full.csv",
        "Chronos-Bolt Mini": "chronosboltmini_results_lag21_full.csv",
        "Chronos-Bolt Small": "chronosboltsmall_results_lag21_full.csv",
        "TimesFM 1.0-200M": "timesfm1_results_lag21_full.csv",
        "TimesFM 2.0-500M": "timesfm2_results_lag21_full.csv",
        "Moirai Small": "uni2tssmall_results_lag21_full.csv",
        "Moirai-Moe Small": "uni2tssmallmoe_results_lag21_full.csv",
        "Moirai-Moe Base": "uni2tsbasemoe_results_lag21_full.csv"
    }

    print("DIEBOLD-MARIANO TEST")
    print("="*60)
    print("Loss Function: Mean Squared Error (MSE)")
    print("H₀: E[dₜ] = 0 (equal prediction accuracy)")
    print("H₁: E[dₜ] ≠ 0 (unequal prediction accuracy)")
    print("Variance Estimator: Newey-West with automatic bandwidth")
    print("="*60)

    # Load and align data
    dfs = []
    model_names = []

    for model_name, file_name in model_files.items():
        file_path = os.path.join(DATA_DIR, file_name)
        try:
            df = pd.read_csv(file_path)

            # Standardize column names
            if 'excess_ret' in df.columns:
                df = df.rename(columns={'excess_ret': 'actual'})
            elif 'y_true' in df.columns:
                df = df.rename(columns={'y_true': 'actual'})

            if 'y_pred' in df.columns:
                df = df.rename(columns={'y_pred': 'predicted'})
            elif 'predicted_excess_returns_lag21' in df.columns:
                df = df.rename(columns={'predicted_excess_returns_lag21': 'predicted'})

            dfs.append(df[['actual', 'predicted']])
            model_names.append(model_name)
            print(f"Loaded: {model_name}")

        except FileNotFoundError:
            print(f"Warning: {file_path} not found, skipping {model_name}")

    # Align to common length
    min_length = min(len(df) for df in dfs)
    print(f"\nSample size after alignment: {min_length}")

    actual_values = dfs[0]['actual'].iloc[:min_length].values
    predictions = {}

    for df, name in zip(dfs, model_names):
        predictions[name] = df['predicted'].iloc[:min_length].values

    print(f"Models included: {len(model_names)}")

    # Run DM tests
    dm_matrix, p_matrix = run_pairwise_dm_tests(actual_values, predictions)

    # Format results
    formatted_dm = format_dm_table(dm_matrix, p_matrix)
    formatted_p = format_pvalue_table(p_matrix)

    print("\nDM TEST STATISTICS:")
    print("Interpretation:")
    print("- DM > 0: Row model has HIGHER MSE (worse) than Column model")
    print("- DM < 0: Row model has LOWER MSE (better) than Column model")
    print("- Significance: *** p<0.001, ** p<0.01, * p<0.05")
    print("- p > 0.05: No significant difference (accept H₀)")
    print("-" * 80)
    print(formatted_dm.to_string())

    print("\n\nP-VALUES:")
    print("- p < 0.05: Reject H₀ (significant difference in accuracy)")
    print("- p ≥ 0.05: Accept H₀ (no significant difference)")
    print("-" * 80)
    print(formatted_p.to_string())

    # Save the DM test statistics results to a CSV file
    formatted_dm.to_csv("dm_statistics_formatted.csv", index=True)
    p_matrix.to_csv("dm_pvalues_formatted.csv", index=True)

    # Create summary of best performing models
    mse_performance = {}
    for name, preds in predictions.items():
        mse = np.mean((actual_values - preds)**2)
        mse_performance[name] = mse

    # Summary of significant outperformances
    print("\n\nSUMMARY OF SIGNIFICANT OUTPERFORMANCES:")
    print("-" * 50)

    significant_better = {}
    for i, model_a in enumerate(model_names):
        significant_better[model_a] = []
        for j, model_b in enumerate(model_names):
            if i != j:
                dm_stat = dm_matrix.iloc[i, j]
                p_val = p_matrix.iloc[i, j]

                # Model A significantly better than Model B (DM < 0 and p < 0.05)
                if not np.isnan(p_val) and p_val < 0.05 and dm_stat < 0:
                    significant_better[model_a].append(model_b)

    for model, beaten_models in significant_better.items():
        if beaten_models:
            print(f"{model} significantly outperforms: {', '.join(beaten_models[:5])}")
            if len(beaten_models) > 5:
                print(f"   ... and {len(beaten_models)-5} more models")

    # Create and display heatmap
    print("\nGenerating DM test heatmap...")
    create_dm_heatmap(p_matrix)

    print(f"\nResults saved:")
    print("- dm_statistics21.csv")
    print("- dm_pvalues.csv")
    print("- dm_statistics_formatted21.csv")
    print("- dm_pvalues_formatted.csv")
    print("- dm_test_heatmap_fixed.png")

    return dm_matrix, p_matrix

def run_pairwise_dm_tests(actual, predictions_dict):
    """
    Run pairwise DM tests for all model combinations
    """
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)

    # Initialize matrices
    dm_stats = np.full((n_models, n_models), np.nan)
    p_values = np.full((n_models, n_models), np.nan)

    print(f"Running pairwise DM tests...")

    # Pairwise comparisons
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:  # Skip diagonal
                pred1 = predictions_dict[model_a]
                pred2 = predictions_dict[model_b]

                dm_stat, p_value = diebold_mariano_test(actual, pred1, pred2)

                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_value

    # Create DataFrames
    dm_df = pd.DataFrame(dm_stats, index=model_names, columns=model_names)
    p_df = pd.DataFrame(p_values, index=model_names, columns=model_names)

    return dm_df, p_df

def create_dm_heatmap(p_matrix, title="Diebold-Mariano Test P-Values"):
    """
    Create heatmap with black diagonal X marks and pure black for p >= 0.05
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Prepare data matrix
    n_models = len(p_matrix)
    heatmap_data = np.full((n_models, n_models), np.nan)

    # Fill data: 0.051 for diagonal and p>=0.05, actual p-values for p<0.05
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
            else:
                p_val = p_matrix.iloc[i, j]
                if np.isnan(p_val) or p_val >= 0.05:
                    heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
                else:
                    heatmap_data[i, j] = p_val

    # Create discrete colormap with explicit black at the end
    colors = ['#006400', '#007400', '#009900', '#00CC00', '#33FF33', '#66FF66', '#99FF99',
              '#CCFF99', '#FFFF66', '#FFCC33', '#FF9900', '#FF6600', '#FF3300', '#CC0000', '#000000']

    # Create boundaries for discrete colors
    boundaries = np.linspace(0, 0.05, len(colors)-1).tolist() + [0.051, 1.0]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries, len(colors))

    # Plot heatmap
    im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='equal', interpolation='nearest')

    # Add white X marks on diagonal
    for i in range(n_models):
        ax.text(i, i, 'X', ha='center', va='center',
                color='white', fontsize=18, fontweight='bold')

    # Add white grid lines
    for i in range(n_models + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1.0)
        ax.axvline(i - 0.5, color='white', linewidth=1.0)

    # Set labels
    model_names = p_matrix.columns.tolist()
    ax.set_xticks(range(n_models))
    ax.set_yticks(range(n_models))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=11, color='black')
    ax.set_yticklabels(model_names, fontsize=11, color='black')

    # Add title
    ax.set_title(title + '\n(Window 21)', fontsize=18, fontweight='bold', color='black', pad=30)

    # Create colorbar with discrete ticks including black
    cbar = plt.colorbar(im, ax=ax, shrink=0.7, aspect=25, pad=0.02, boundaries=boundaries, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.051])
    cbar.set_label('P-Value', fontsize=14, color='black', labelpad=15)
    cbar.ax.tick_params(labelsize=12, colors='black')

    # Set colorbar labels with black indicator
    cbar_labels = ['0.00', '0.01', '0.02', '0.03', '0.04', '0.05', '≥0.05']
    cbar.set_ticklabels(cbar_labels)

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove tick marks but keep labels
    ax.tick_params(length=0, colors='black')

    # Adjust layout
    plt.subplots_adjust(bottom=0.08, top=0.92, left=0.1, right=0.88)

    plt.savefig('dm_test_heatmap_fixed.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()

    return ax

# Run the analysis
if __name__ == "__main__":
    dm_df, p_df = run_dm_test_analysis()

# Window Size 252

In [ ]:
def diebold_mariano_test(actual, pred1, pred2, h=1):
    """
    Diebold-Mariano test using Mean Squared Error loss function with Newey-West

    H0: E[dt] = 0 (equal prediction accuracy)
    H1: E[dt] ≠ 0 (unequal prediction accuracy)

    Test statistic: DM = d̄ / σ̂_d̄ ~ N(0,1)
    """

    # Align arrays
    min_len = min(len(actual), len(pred1), len(pred2))
    actual = np.array(actual)[:min_len]
    pred1 = np.array(pred1)[:min_len]
    pred2 = np.array(pred2)[:min_len]

    # Remove missing values
    mask = ~(np.isnan(actual) | np.isnan(pred1) | np.isnan(pred2))
    actual = actual[mask]
    pred1 = pred1[mask]
    pred2 = pred2[mask]

    if len(actual) < 10:
        return np.nan, np.nan

    # Calculate squared forecast errors
    e1_squared = (actual - pred1)**2
    e2_squared = (actual - pred2)**2

    # Loss differential: dt = e²1,t - e²2,t
    d = e1_squared - e2_squared
    d_mean = np.mean(d)
    n = len(d)

    # Newey-West variance estimator
    def newey_west_variance(d, h):
        n = len(d)
        d_centered = d - np.mean(d)

        # Bandwidth selection
        bandwidth = max(1, int(4 * (n/100)**(2/9)))

        # Calculate autocovariances with Bartlett kernel
        gamma_0 = np.mean(d_centered**2)
        gamma_sum = 0

        for k in range(1, min(bandwidth + 1, n)):
            if k < n:
                weight = 1 - k / (bandwidth + 1)
                gamma_k = np.mean(d_centered[k:] * d_centered[:-k])
                gamma_sum += 2 * weight * gamma_k

        return gamma_0 + gamma_sum

    # Calculate long-run variance
    long_run_var = newey_west_variance(d, h)

    if long_run_var <= 0:
        long_run_var = np.var(d, ddof=1)
        if long_run_var <= 0:
            return np.nan, np.nan

    # DM statistic
    dm_stat = d_mean / np.sqrt(long_run_var / n)

    if np.isnan(dm_stat) or np.isinf(dm_stat):
        return np.nan, np.nan

    # Two-tailed p-value
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))

    return dm_stat, p_value


def run_pairwise_dm_tests(actual, predictions_dict):
    """
    Run pairwise DM tests for all model combinations
    """
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)

    # Initialize matrices
    dm_stats = np.full((n_models, n_models), np.nan)
    p_values = np.full((n_models, n_models), np.nan)

    print(f"Running pairwise DM tests...")

    # Pairwise comparisons
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:  # Skip diagonal
                pred1 = predictions_dict[model_a]
                pred2 = predictions_dict[model_b]

                dm_stat, p_value = diebold_mariano_test(actual, pred1, pred2)

                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_value

    # Create DataFrames
    dm_df = pd.DataFrame(dm_stats, index=model_names, columns=model_names)
    p_df = pd.DataFrame(p_values, index=model_names, columns=model_names)

    return dm_df, p_df


def format_pvalue_table(p_df):
    """Format p-values with significance indicators"""
    formatted = p_df.copy().astype(object)

    for i in range(len(p_df)):
        for j in range(len(p_df.columns)):
            if i == j:
                formatted.iloc[i, j] = "-"
            elif i > j:  # Lower triangle - leave empty
                formatted.iloc[i, j] = ""
            elif not np.isnan(p_df.iloc[i, j]):
                p_val = p_df.iloc[i, j]

                if p_val < 0.001:
                    formatted.iloc[i, j] = f"{p_val:.6f}***"
                elif p_val < 0.01:
                    formatted.iloc[i, j] = f"{p_val:.6f}**"
                elif p_val < 0.05:
                    formatted.iloc[i, j] = f"{p_val:.6f}*"
                else:
                    formatted.iloc[i, j] = f"{p_val:.6f}"
            else:
                formatted.iloc[i, j] = ""

    return formatted


def format_dm_table(dm_df, p_df):
    """Format DM statistics table with significance indicators based on p-values"""
    formatted = dm_df.copy().astype(object)

    for i in range(len(dm_df)):
        for j in range(len(dm_df.columns)):
            if i == j:
                formatted.iloc[i, j] = "-"
            elif i > j:
                formatted.iloc[i, j] = ""
            elif not (np.isnan(dm_df.iloc[i, j]) or np.isnan(p_df.iloc[i, j])):
                dm_val = dm_df.iloc[i, j]
                p_val = p_df.iloc[i, j]

                # Add significance stars based on p-values
                if p_val < 0.001:
                    stars = "***"
                elif p_val < 0.01:
                    stars = "**"
                elif p_val < 0.05:
                    stars = "*"
                else:
                    stars = ""

                formatted.iloc[i, j] = f"{dm_val:.4f}{stars}"
            else:
                formatted.iloc[i, j] = ""

    return formatted


def create_dm_heatmap(p_matrix, title="Diebold-Mariano Test P-Values"):
    """
    Create heatmap with black diagonal X marks and pure black for p >= 0.05
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Prepare data matrix
    n_models = len(p_matrix)
    heatmap_data = np.full((n_models, n_models), np.nan)

    # Fill data: 0.051 for diagonal and p>=0.05, actual p-values for p<0.05
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
            else:
                p_val = p_matrix.iloc[i, j]
                if np.isnan(p_val) or p_val >= 0.05:
                    heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
                else:
                    heatmap_data[i, j] = p_val  # Keep actual p-value for coloring

    # Create discrete colormap with explicit black at the end
    colors = ['#003300', '#006600', '#009900', '#00CC00', '#33FF33', '#66FF66', '#99FF99',
              '#CCFF99', '#FFFF66', '#FFCC33', '#FF9900', '#FF6600', '#FF3300', '#CC0000', '#000000']

    # Create boundaries for discrete colors
    boundaries = np.linspace(0, 0.05, len(colors)-1).tolist() + [0.051, 1.0]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries, len(colors))

    # Plot heatmap
    im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='equal', interpolation='nearest')

    # Add white X marks on diagonal
    for i in range(n_models):
        ax.text(i, i, 'X', ha='center', va='center',
                color='white', fontsize=18, fontweight='bold')

    # Add white grid lines
    for i in range(n_models + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1.0)
        ax.axvline(i - 0.5, color='white', linewidth=1.0)

    # Set labels
    model_names = p_matrix.columns.tolist()
    ax.set_xticks(range(n_models))
    ax.set_yticks(range(n_models))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=11, color='black')
    ax.set_yticklabels(model_names, fontsize=11, color='black')

    # Add title
    ax.set_title(title + '\n(Window 252)', fontsize=18, fontweight='bold', color='black', pad=30)

    # Create colorbar with discrete ticks including black
    cbar = plt.colorbar(im, ax=ax, shrink=0.7, aspect=25, pad=0.02, boundaries=boundaries, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.051])
    cbar.set_label('P-Value', fontsize=14, color='black', labelpad=15)
    cbar.ax.tick_params(labelsize=12, colors='black')

    # Set colorbar labels with black indicator
    cbar_labels = ['0.00', '0.01', '0.02', '0.03', '0.04', '0.05', '≥0.05']
    cbar.set_ticklabels(cbar_labels)

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove tick marks but keep labels
    ax.tick_params(length=0, colors='black')

    # Adjust layout
    plt.subplots_adjust(bottom=0.08, top=0.92, left=0.1, right=0.88)

    plt.savefig('dm_test_heatmap_fixed.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()

    return ax


def run_dm_test_analysis():
    """
    Load data and run DM tests for all model pairs
    """
    
    # Input folder: DM_Test_Data
    DATA_DIR = os.path.join(os.getcwd(), "DM_Test_Data")

    # Model files
    model_files = {
        "OLS": "ols_results_lag252_full.csv",
        "LASSO": "lasso_results_lag252_full.csv",
        "Ridge": "ridge_results_lag252_full.csv",
        "ElasticNet": "enet_results_lag252_full.csv",
        "PCR": "pcr_results_lag252_full.csv",
        "GLM": "glm_results_lag252_full.csv",
        "RF": "rf_results_lag252_full.csv",
        "GBRT": "gbrt_results_lag252_full.csv",
        "NN1": "nn1_results_lag252_full.csv",
        "NN2": "nn2_results_lag252_full.csv",
        "NN3": "nn3_results_lag252_full.csv",
        "NN4": "nn4_results_lag252_full.csv",
        "NN5": "nn5_results_lag252_full.csv",
        "Chronos-T5 Tiny": "chronost5tiny_results_lag252_full.csv",
        "Chronos-Bolt Tiny": "chronosbolttiny_results_lag252_full.csv",
        "Chronos-Bolt Mini": "chronosboltmini_results_lag252_full.csv",
        "Chronos-Bolt Small": "chronosboltsmall_results_lag252_full.csv",
        "TimesFM 1.0-200M": "timesfm1_results_lag252_full.csv",
        "TimesFM 2.0-500M": "timesfm2_results_lag252_full.csv",
        "Moirai Small": "uni2tssmall_results_lag252_full.csv",
        "Moirai-Moe Small": "uni2tssmallmoe_results_lag252_full.csv",
        "Moirai-Moe Base": "uni2tsbasemoe_results_lag252_full.csv"
    }

    print("DIEBOLD-MARIANO TEST")
    print("="*60)
    print("Loss Function: Mean Squared Error (MSE)")
    print("H₀: E[dₜ] = 0 (equal prediction accuracy)")
    print("H₁: E[dₜ] ≠ 0 (unequal prediction accuracy)")
    print("Variance Estimator: Newey-West with automatic bandwidth")
    print("="*60)

    # Load and align data
    dfs = []
    model_names = []

    for model_name, file_name in model_files.items():
        file_path = os.path.join(DATA_DIR, file_name)
        try:
            df = pd.read_csv(file_path)

            # Standardize column names
            if 'excess_ret' in df.columns:
                df = df.rename(columns={'excess_ret': 'actual'})
            elif 'y_true' in df.columns:
                df = df.rename(columns={'y_true': 'actual'})

            if 'y_pred' in df.columns:
                df = df.rename(columns={'y_pred': 'predicted'})
            elif 'predicted_excess_returns_lag252' in df.columns:
                df = df.rename(columns={'predicted_excess_returns_lag252': 'predicted'})

            dfs.append(df[['actual', 'predicted']])
            model_names.append(model_name)
            print(f"Loaded: {model_name}")

        except FileNotFoundError:
            print(f"Warning: {file_path} not found, skipping {model_name}")

    # Align to common length
    min_length = min(len(df) for df in dfs)
    print(f"\nSample size after alignment: {min_length}")

    actual_values = dfs[0]['actual'].iloc[:min_length].values
    predictions = {}

    for df, name in zip(dfs, model_names):
        predictions[name] = df['predicted'].iloc[:min_length].values

    print(f"Models included: {len(model_names)}")

    # Run DM tests
    dm_matrix, p_matrix = run_pairwise_dm_tests(actual_values, predictions)

    # Format results
    formatted_dm = format_dm_table(dm_matrix, p_matrix)
    formatted_p = format_pvalue_table(p_matrix)

    print("\nDM TEST STATISTICS:")
    print("Interpretation:")
    print("- DM > 0: Row model has HIGHER MSE (worse) than Column model")
    print("- DM < 0: Row model has LOWER MSE (better) than Column model")
    print("- Significance: *** p<0.001, ** p<0.01, * p<0.05")
    print("- p > 0.05: No significant difference (accept H₀)")
    print("-" * 80)
    print(formatted_dm.to_string())

    print("\n\nP-VALUES:")
    print("- p < 0.05: Reject H₀ (significant difference in accuracy)")
    print("- p ≥ 0.05: Accept H₀ (no significant difference)")
    print("-" * 80)
    print(formatted_p.to_string())

    # Save the DM test statistics results to a CSV file
    formatted_dm.to_csv("dm_statistics_formatted.csv", index=True)
    p_matrix.to_csv("dm_pvalues_formatted.csv", index=True)

    # Create summary of best performing models
    mse_performance = {}
    for name, preds in predictions.items():
        mse = np.mean((actual_values - preds)**2)
        mse_performance[name] = mse

    # Summary of significant outperformances
    print("\n\nSUMMARY OF SIGNIFICANT OUTPERFORMANCES:")
    print("-" * 50)

    significant_better = {}
    for i, model_a in enumerate(model_names):
        significant_better[model_a] = []
        for j, model_b in enumerate(model_names):
            if i != j:
                dm_stat = dm_matrix.iloc[i, j]
                p_val = p_matrix.iloc[i, j]

                # Model A significantly better than Model B (DM < 0 and p < 0.05)
                if not np.isnan(p_val) and p_val < 0.05 and dm_stat < 0:
                    significant_better[model_a].append(model_b)

    for model, beaten_models in significant_better.items():
        if beaten_models:
            print(f"{model} significantly outperforms: {', '.join(beaten_models[:5])}")
            if len(beaten_models) > 5:
                print(f"   ... and {len(beaten_models)-5} more models")

    # Create and display heatmap
    print("\nGenerating DM test heatmap...")
    create_dm_heatmap(p_matrix)

    print(f"\nResults saved:")
    print("- dm_statistics252.csv")
    print("- dm_pvalues.csv")
    print("- dm_statistics_formatted252.csv")
    print("- dm_pvalues_formatted.csv")
    print("- dm_test_heatmap_fixed.png")

    return dm_matrix, p_matrix

def run_pairwise_dm_tests(actual, predictions_dict):
    """
    Run pairwise DM tests for all model combinations
    """
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)

    # Initialize matrices
    dm_stats = np.full((n_models, n_models), np.nan)
    p_values = np.full((n_models, n_models), np.nan)

    print(f"Running pairwise DM tests...")

    # Pairwise comparisons
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:  # Skip diagonal
                pred1 = predictions_dict[model_a]
                pred2 = predictions_dict[model_b]

                dm_stat, p_value = diebold_mariano_test(actual, pred1, pred2)

                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_value

    # Create DataFrames
    dm_df = pd.DataFrame(dm_stats, index=model_names, columns=model_names)
    p_df = pd.DataFrame(p_values, index=model_names, columns=model_names)

    return dm_df, p_df

def create_dm_heatmap(p_matrix, title="Diebold-Mariano Test P-Values"):
    """
    Create heatmap with black diagonal X marks and pure black for p >= 0.05
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Prepare data matrix
    n_models = len(p_matrix)
    heatmap_data = np.full((n_models, n_models), np.nan)

    # Fill data: 0.051 for diagonal and p>=0.05, actual p-values for p<0.05
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
            else:
                p_val = p_matrix.iloc[i, j]
                if np.isnan(p_val) or p_val >= 0.05:
                    heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
                else:
                    heatmap_data[i, j] = p_val

    # Create discrete colormap with explicit black at the end
    colors = ['#006400', '#007400', '#009900', '#00CC00', '#33FF33', '#66FF66', '#99FF99',
              '#CCFF99', '#FFFF66', '#FFCC33', '#FF9900', '#FF6600', '#FF3300', '#CC0000', '#000000']

    # Create boundaries for discrete colors
    boundaries = np.linspace(0, 0.05, len(colors)-1).tolist() + [0.051, 1.0]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries, len(colors))

    # Plot heatmap
    im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='equal', interpolation='nearest')

    # Add white X marks on diagonal
    for i in range(n_models):
        ax.text(i, i, 'X', ha='center', va='center',
                color='white', fontsize=18, fontweight='bold')

    # Add white grid lines
    for i in range(n_models + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1.0)
        ax.axvline(i - 0.5, color='white', linewidth=1.0)

    # Set labels
    model_names = p_matrix.columns.tolist()
    ax.set_xticks(range(n_models))
    ax.set_yticks(range(n_models))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=11, color='black')
    ax.set_yticklabels(model_names, fontsize=11, color='black')

    # Add title
    ax.set_title(title + '\n(Window 252)', fontsize=18, fontweight='bold', color='black', pad=30)

    # Create colorbar with discrete ticks including black
    cbar = plt.colorbar(im, ax=ax, shrink=0.7, aspect=25, pad=0.02, boundaries=boundaries, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.051])
    cbar.set_label('P-Value', fontsize=14, color='black', labelpad=15)
    cbar.ax.tick_params(labelsize=12, colors='black')

    # Set colorbar labels with black indicator
    cbar_labels = ['0.00', '0.01', '0.02', '0.03', '0.04', '0.05', '≥0.05']
    cbar.set_ticklabels(cbar_labels)

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove tick marks but keep labels
    ax.tick_params(length=0, colors='black')

    # Adjust layout
    plt.subplots_adjust(bottom=0.08, top=0.92, left=0.1, right=0.88)

    plt.savefig('dm_test_heatmap_fixed.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()

    return ax

# Run the analysis
if __name__ == "__main__":
    dm_df, p_df = run_dm_test_analysis()

# Window Size 512

In [ ]:
def diebold_mariano_test(actual, pred1, pred2, h=1):
    """
    Diebold-Mariano test using Mean Squared Error loss function with Newey-West

    H0: E[dt] = 0 (equal prediction accuracy)
    H1: E[dt] ≠ 0 (unequal prediction accuracy)

    Test statistic: DM = d̄ / σ̂_d̄ ~ N(0,1)
    """

    # Align arrays
    min_len = min(len(actual), len(pred1), len(pred2))
    actual = np.array(actual)[:min_len]
    pred1 = np.array(pred1)[:min_len]
    pred2 = np.array(pred2)[:min_len]

    # Remove missing values
    mask = ~(np.isnan(actual) | np.isnan(pred1) | np.isnan(pred2))
    actual = actual[mask]
    pred1 = pred1[mask]
    pred2 = pred2[mask]

    if len(actual) < 10:
        return np.nan, np.nan

    # Calculate squared forecast errors
    e1_squared = (actual - pred1)**2
    e2_squared = (actual - pred2)**2

    # Loss differential: dt = e²1,t - e²2,t
    d = e1_squared - e2_squared
    d_mean = np.mean(d)
    n = len(d)

    # Newey-West variance estimator
    def newey_west_variance(d, h):
        n = len(d)
        d_centered = d - np.mean(d)

        # Bandwidth selection
        bandwidth = max(1, int(4 * (n/100)**(2/9)))

        # Calculate autocovariances with Bartlett kernel
        gamma_0 = np.mean(d_centered**2)
        gamma_sum = 0

        for k in range(1, min(bandwidth + 1, n)):
            if k < n:
                weight = 1 - k / (bandwidth + 1)
                gamma_k = np.mean(d_centered[k:] * d_centered[:-k])
                gamma_sum += 2 * weight * gamma_k

        return gamma_0 + gamma_sum

    # Calculate long-run variance
    long_run_var = newey_west_variance(d, h)

    if long_run_var <= 0:
        long_run_var = np.var(d, ddof=1)
        if long_run_var <= 0:
            return np.nan, np.nan

    # DM statistic
    dm_stat = d_mean / np.sqrt(long_run_var / n)

    if np.isnan(dm_stat) or np.isinf(dm_stat):
        return np.nan, np.nan

    # Two-tailed p-value
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))

    return dm_stat, p_value


def run_pairwise_dm_tests(actual, predictions_dict):
    """
    Run pairwise DM tests for all model combinations
    """
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)

    # Initialize matrices
    dm_stats = np.full((n_models, n_models), np.nan)
    p_values = np.full((n_models, n_models), np.nan)

    print(f"Running pairwise DM tests...")

    # Pairwise comparisons
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:  # Skip diagonal
                pred1 = predictions_dict[model_a]
                pred2 = predictions_dict[model_b]

                dm_stat, p_value = diebold_mariano_test(actual, pred1, pred2)

                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_value

    # Create DataFrames
    dm_df = pd.DataFrame(dm_stats, index=model_names, columns=model_names)
    p_df = pd.DataFrame(p_values, index=model_names, columns=model_names)

    return dm_df, p_df


def format_pvalue_table(p_df):
    """Format p-values with significance indicators"""
    formatted = p_df.copy().astype(object)

    for i in range(len(p_df)):
        for j in range(len(p_df.columns)):
            if i == j:
                formatted.iloc[i, j] = "-"
            elif i > j:  # Lower triangle - leave empty
                formatted.iloc[i, j] = ""
            elif not np.isnan(p_df.iloc[i, j]):
                p_val = p_df.iloc[i, j]

                if p_val < 0.001:
                    formatted.iloc[i, j] = f"{p_val:.6f}***"
                elif p_val < 0.01:
                    formatted.iloc[i, j] = f"{p_val:.6f}**"
                elif p_val < 0.05:
                    formatted.iloc[i, j] = f"{p_val:.6f}*"
                else:
                    formatted.iloc[i, j] = f"{p_val:.6f}"
            else:
                formatted.iloc[i, j] = ""

    return formatted


def format_dm_table(dm_df, p_df):
    """Format DM statistics table with significance indicators based on p-values"""
    formatted = dm_df.copy().astype(object)

    for i in range(len(dm_df)):
        for j in range(len(dm_df.columns)):
            if i == j:
                formatted.iloc[i, j] = "-"
            elif i > j:
                formatted.iloc[i, j] = ""
            elif not (np.isnan(dm_df.iloc[i, j]) or np.isnan(p_df.iloc[i, j])):
                dm_val = dm_df.iloc[i, j]
                p_val = p_df.iloc[i, j]

                # Add significance stars based on p-values
                if p_val < 0.001:
                    stars = "***"
                elif p_val < 0.01:
                    stars = "**"
                elif p_val < 0.05:
                    stars = "*"
                else:
                    stars = ""

                formatted.iloc[i, j] = f"{dm_val:.4f}{stars}"
            else:
                formatted.iloc[i, j] = ""

    return formatted


def create_dm_heatmap(p_matrix, title="Diebold-Mariano Test P-Values"):
    """
    Create heatmap with black diagonal X marks and pure black for p >= 0.05
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Prepare data matrix
    n_models = len(p_matrix)
    heatmap_data = np.full((n_models, n_models), np.nan)

    # Fill data: 0.051 for diagonal and p>=0.05, actual p-values for p<0.05
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
            else:
                p_val = p_matrix.iloc[i, j]
                if np.isnan(p_val) or p_val >= 0.05:
                    heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
                else:
                    heatmap_data[i, j] = p_val  # Keep actual p-value for coloring

    # Create discrete colormap with explicit black at the end
    colors = ['#003300', '#006600', '#009900', '#00CC00', '#33FF33', '#66FF66', '#99FF99',
              '#CCFF99', '#FFFF66', '#FFCC33', '#FF9900', '#FF6600', '#FF3300', '#CC0000', '#000000']

    # Create boundaries for discrete colors
    boundaries = np.linspace(0, 0.05, len(colors)-1).tolist() + [0.051, 1.0]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries, len(colors))

    # Plot heatmap
    im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='equal', interpolation='nearest')

    # Add white X marks on diagonal
    for i in range(n_models):
        ax.text(i, i, 'X', ha='center', va='center',
                color='white', fontsize=18, fontweight='bold')

    # Add white grid lines
    for i in range(n_models + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1.0)
        ax.axvline(i - 0.5, color='white', linewidth=1.0)

    # Set labels
    model_names = p_matrix.columns.tolist()
    ax.set_xticks(range(n_models))
    ax.set_yticks(range(n_models))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=11, color='black')
    ax.set_yticklabels(model_names, fontsize=11, color='black')

    # Add title
    ax.set_title(title + '\n(Window 512)', fontsize=18, fontweight='bold', color='black', pad=30)

    # Create colorbar with discrete ticks including black
    cbar = plt.colorbar(im, ax=ax, shrink=0.7, aspect=25, pad=0.02, boundaries=boundaries, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.051])
    cbar.set_label('P-Value', fontsize=14, color='black', labelpad=15)
    cbar.ax.tick_params(labelsize=12, colors='black')

    # Set colorbar labels with black indicator
    cbar_labels = ['0.00', '0.01', '0.02', '0.03', '0.04', '0.05', '≥0.05']
    cbar.set_ticklabels(cbar_labels)

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove tick marks but keep labels
    ax.tick_params(length=0, colors='black')

    # Adjust layout
    plt.subplots_adjust(bottom=0.08, top=0.92, left=0.1, right=0.88)

    plt.savefig('dm_test_heatmap_fixed.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()

    return ax


def run_dm_test_analysis():
    """
    Load data and run DM tests for all model pairs
    """

    # Input folder: DM_Test_Data
    DATA_DIR = os.path.join(os.getcwd(), "DM_Test_Data")
    
    # Model files
    model_files = {
        "OLS": "ols_results_lag512_full.csv",
        "LASSO": "lasso_results_lag512_full.csv",
        "Ridge": "ridge_results_lag512_full.csv",
        "ElasticNet": "enet_results_lag512_full.csv",
        "PCR": "pcr_results_lag512_full.csv",
        "GLM": "glm_results_lag512_full.csv",
        "RF": "rf_results_lag512_full.csv",
        "GBRT": "gbrt_results_lag512_full.csv",
        "NN1": "nn1_results_lag512_full.csv",
        "NN2": "nn2_results_lag512_full.csv",
        "NN3": "nn3_results_lag512_full.csv",
        "NN4": "nn4_results_lag512_full.csv",
        "NN5": "nn5_results_lag512_full.csv",
        "Chronos-T5 Tiny": "chronost5tiny_results_lag512_full.csv",
        "Chronos-Bolt Tiny": "chronosbolttiny_results_lag512_full.csv",
        "Chronos-Bolt Mini": "chronosboltmini_results_lag512_full.csv",
        "Chronos-Bolt Small": "chronosboltsmall_results_lag512_full.csv",
        "TimesFM 1.0-200M": "timesfm1_results_lag512_full.csv",
        "TimesFM 2.0-500M": "timesfm2_results_lag512_full.csv",
        "Moirai Small": "uni2tssmall_results_lag512_full.csv",
        "Moirai-Moe Small": "uni2tssmallmoe_results_lag512_full.csv",
        "Moirai-Moe Base": "uni2tsbasemoe_results_lag512_full.csv"
    }

    print("DIEBOLD-MARIANO TEST")
    print("="*60)
    print("Loss Function: Mean Squared Error (MSE)")
    print("H₀: E[dₜ] = 0 (equal prediction accuracy)")
    print("H₁: E[dₜ] ≠ 0 (unequal prediction accuracy)")
    print("Variance Estimator: Newey-West with automatic bandwidth")
    print("="*60)

    # Load and align data
    dfs = []
    model_names = []

    for model_name, file_name in model_files.items():
        file_path = os.path.join(DATA_DIR, file_name)
        try:
            df = pd.read_csv(file_path)

            # Standardize column names
            if 'excess_ret' in df.columns:
                df = df.rename(columns={'excess_ret': 'actual'})
            elif 'y_true' in df.columns:
                df = df.rename(columns={'y_true': 'actual'})

            if 'y_pred' in df.columns:
                df = df.rename(columns={'y_pred': 'predicted'})
            elif 'predicted_excess_returns_lag512' in df.columns:
                df = df.rename(columns={'predicted_excess_returns_lag512': 'predicted'})

            dfs.append(df[['actual', 'predicted']])
            model_names.append(model_name)
            print(f"Loaded: {model_name}")

        except FileNotFoundError:
            print(f"Warning: {file_path} not found, skipping {model_name}")

    # Align to common length
    min_length = min(len(df) for df in dfs)
    print(f"\nSample size after alignment: {min_length}")

    actual_values = dfs[0]['actual'].iloc[:min_length].values
    predictions = {}

    for df, name in zip(dfs, model_names):
        predictions[name] = df['predicted'].iloc[:min_length].values

    print(f"Models included: {len(model_names)}")

    # Run DM tests
    dm_matrix, p_matrix = run_pairwise_dm_tests(actual_values, predictions)

    # Format results
    formatted_dm = format_dm_table(dm_matrix, p_matrix)
    formatted_p = format_pvalue_table(p_matrix)

    print("\nDM TEST STATISTICS:")
    print("Interpretation:")
    print("- DM > 0: Row model has HIGHER MSE (worse) than Column model")
    print("- DM < 0: Row model has LOWER MSE (better) than Column model")
    print("- Significance: *** p<0.001, ** p<0.01, * p<0.05")
    print("- p > 0.05: No significant difference (accept H₀)")
    print("-" * 80)
    print(formatted_dm.to_string())

    print("\n\nP-VALUES:")
    print("- p < 0.05: Reject H₀ (significant difference in accuracy)")
    print("- p ≥ 0.05: Accept H₀ (no significant difference)")
    print("-" * 80)
    print(formatted_p.to_string())

    # Save the DM test statistics results to a CSV file
    formatted_dm.to_csv("dm_statistics_formatted.csv", index=True)
    p_matrix.to_csv("dm_pvalues_formatted.csv", index=True)

    # Create summary of best performing models
    mse_performance = {}
    for name, preds in predictions.items():
        mse = np.mean((actual_values - preds)**2)
        mse_performance[name] = mse

    # Summary of significant outperformances
    print("\n\nSUMMARY OF SIGNIFICANT OUTPERFORMANCES:")
    print("-" * 50)

    significant_better = {}
    for i, model_a in enumerate(model_names):
        significant_better[model_a] = []
        for j, model_b in enumerate(model_names):
            if i != j:
                dm_stat = dm_matrix.iloc[i, j]
                p_val = p_matrix.iloc[i, j]

                # Model A significantly better than Model B (DM < 0 and p < 0.05)
                if not np.isnan(p_val) and p_val < 0.05 and dm_stat < 0:
                    significant_better[model_a].append(model_b)

    for model, beaten_models in significant_better.items():
        if beaten_models:
            print(f"{model} significantly outperforms: {', '.join(beaten_models[:5])}")
            if len(beaten_models) > 5:
                print(f"   ... and {len(beaten_models)-5} more models")

    # Create and display heatmap
    print("\nGenerating DM test heatmap...")
    create_dm_heatmap(p_matrix)

    print(f"\nResults saved:")
    print("- dm_statistics512.csv")
    print("- dm_pvalues.csv")
    print("- dm_statistics_formatted512.csv")
    print("- dm_pvalues_formatted.csv")
    print("- dm_test_heatmap_fixed.png")

    return dm_matrix, p_matrix

def run_pairwise_dm_tests(actual, predictions_dict):
    """
    Run pairwise DM tests for all model combinations
    """
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)

    # Initialize matrices
    dm_stats = np.full((n_models, n_models), np.nan)
    p_values = np.full((n_models, n_models), np.nan)

    print(f"Running pairwise DM tests...")

    # Pairwise comparisons
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:  # Skip diagonal
                pred1 = predictions_dict[model_a]
                pred2 = predictions_dict[model_b]

                dm_stat, p_value = diebold_mariano_test(actual, pred1, pred2)

                dm_stats[i, j] = dm_stat
                p_values[i, j] = p_value

    # Create DataFrames
    dm_df = pd.DataFrame(dm_stats, index=model_names, columns=model_names)
    p_df = pd.DataFrame(p_values, index=model_names, columns=model_names)

    return dm_df, p_df

def create_dm_heatmap(p_matrix, title="Diebold-Mariano Test P-Values"):
    """
    Create heatmap with black diagonal X marks and pure black for p >= 0.05
    """
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Prepare data matrix
    n_models = len(p_matrix)
    heatmap_data = np.full((n_models, n_models), np.nan)

    # Fill data: 0.051 for diagonal and p>=0.05, actual p-values for p<0.05
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
            else:
                p_val = p_matrix.iloc[i, j]
                if np.isnan(p_val) or p_val >= 0.05:
                    heatmap_data[i, j] = 0.051  # Slightly above 0.05 for black
                else:
                    heatmap_data[i, j] = p_val

    # Create discrete colormap with explicit black at the end
    colors = ['#006400', '#007400', '#009900', '#00CC00', '#33FF33', '#66FF66', '#99FF99',
              '#CCFF99', '#FFFF66', '#FFCC33', '#FF9900', '#FF6600', '#FF3300', '#CC0000', '#000000']

    # Create boundaries for discrete colors
    boundaries = np.linspace(0, 0.05, len(colors)-1).tolist() + [0.051, 1.0]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries, len(colors))

    # Plot heatmap
    im = ax.imshow(heatmap_data, cmap=cmap, norm=norm, aspect='equal', interpolation='nearest')

    # Add white X marks on diagonal
    for i in range(n_models):
        ax.text(i, i, 'X', ha='center', va='center',
                color='white', fontsize=18, fontweight='bold')

    # Add white grid lines
    for i in range(n_models + 1):
        ax.axhline(i - 0.5, color='white', linewidth=1.0)
        ax.axvline(i - 0.5, color='white', linewidth=1.0)

    # Set labels
    model_names = p_matrix.columns.tolist()
    ax.set_xticks(range(n_models))
    ax.set_yticks(range(n_models))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=11, color='black')
    ax.set_yticklabels(model_names, fontsize=11, color='black')

    # Add title
    ax.set_title(title + '\n(Window 512)', fontsize=18, fontweight='bold', color='black', pad=30)

    # Create colorbar with discrete ticks including black
    cbar = plt.colorbar(im, ax=ax, shrink=0.7, aspect=25, pad=0.02, boundaries=boundaries, ticks=[0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.051])
    cbar.set_label('P-Value', fontsize=14, color='black', labelpad=15)
    cbar.ax.tick_params(labelsize=12, colors='black')

    # Set colorbar labels with black indicator
    cbar_labels = ['0.00', '0.01', '0.02', '0.03', '0.04', '0.05', '≥0.05']
    cbar.set_ticklabels(cbar_labels)

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove tick marks but keep labels
    ax.tick_params(length=0, colors='black')

    # Adjust layout
    plt.subplots_adjust(bottom=0.08, top=0.92, left=0.1, right=0.88)

    plt.savefig('dm_test_heatmap_fixed.png', dpi=300, bbox_inches='tight',
                facecolor='white', edgecolor='none')
    plt.show()

    return ax

# Run the analysis
if __name__ == "__main__":
    dm_df, p_df = run_dm_test_analysis()